# **Assignment 6_Extra_Credit_mnist_TFX_pipeline**

a colab entirely in tfx interactive context for doing any of the following end2end with all artifacts for mnist dataset

Reference: https://github.com/tensorflow/tfx/tree/master/tfx/examples/mnist

In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

     |████████████████████████████████| 1.5MB 23.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


## **First we need to Install TFX to do this project:**
The runtime should be restarted if we run this cell first time- if not this may show an error!

In [2]:
!pip install -q -U --use-feature=2020-resolver tfx

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from typing import List, Text

import absl



In [4]:
import tensorflow as tf
import tfx

In [5]:
import tensorflow_model_analysis as tfma

In [6]:
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ImportExampleGen
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.components.base import executor_spec
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input

In [7]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.3.0
TFX version: 0.25.0


In [8]:
_pipeline_name = 'mnist_native_keras'

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
path_to_mnist = '/content/drive/MyDrive/Fall_2020/Advanced Deep Learning- CMPE 297- 49/Assignments/Assignment 6_TFX interactivecontext based colab/tfx/examples'

In [11]:
# This example assumes that MNIST data is stored in ~/mnist/data and the utility
# function is in ~/mnist. Feel free to customize as needed.
_mnist_root = os.path.join(path_to_mnist, 'mnist')
_data_root = os.path.join(_mnist_root, 'data')

In [12]:
# Python module files to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_module_file = os.path.join(_mnist_root, 'mnist_utils_native_keras.py')
_module_file_lite = os.path.join(
    _mnist_root, 'mnist_utils_native_keras_lite.py')

In [13]:
# Path which can be listened to by the model server. Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_mnist_root, 'serving_model', _pipeline_name)
_serving_model_dir_lite = os.path.join(
    _mnist_root, 'serving_model_lite', _pipeline_name)

In [14]:
path_to_tfx = '/content/drive/MyDrive/Fall_2020/Advanced Deep Learning- CMPE 297- 49/Assignments/Assignment 6_TFX interactivecontext based colab'

In [15]:
# Directory and data locations.  This example assumes all of the images,
# example code, and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(path_to_tfx, 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
# Sqlite ML-metadata db path.
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

In [16]:
# Pipeline arguments for Beam powered Components.
_beam_pipeline_args = [
    '--direct_running_mode=multi_processing',
    # 0 means auto-detect based on on the number of CPUs available
    # during execution time.
    '--direct_num_workers=0',
]

In [17]:
def _create_pipeline(pipeline_name: Text, pipeline_root: Text, data_root: Text,
                     module_file: Text, module_file_lite: Text,
                     serving_model_dir: Text, serving_model_dir_lite: Text,
                     metadata_path: Text,
                     beam_pipeline_args: List[Text]) -> pipeline.Pipeline:
  """Implements the handwritten digit classification example using TFX."""
  examples = external_input(data_root)

  # Brings data into the pipeline.
  example_gen = ImportExampleGen(input=examples)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

  # Generates schema based on statistics files.
  schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  # Performs anomaly detection based on statistics and data schema.
  example_validator = ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema'])

  # Performs transformations and feature engineering in training and serving.
  transform = Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=module_file)
  def _create_trainer(module_file, instance_name):
    return Trainer(
        module_file=module_file,
        custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(num_steps=5000),
        eval_args=trainer_pb2.EvalArgs(num_steps=100),
        instance_name=instance_name)

  # Uses user-provided Python function that trains a Keras model.
  trainer = _create_trainer(module_file, 'mnist')

  # Trains the same model as the one above, but converts it into a TFLite one.
  trainer_lite = _create_trainer(module_file_lite, 'mnist_lite')

  # TODO(b/150949276): Add resolver back once it supports two trainers.

  # Uses TFMA to compute an evaluation statistics over features of a model and
  # performs quality validation of a candidate model.
  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='image_class')],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='SparseCategoricalAccuracy',
                  threshold=tfma.config.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.8})))
          ])
      ])

  eval_config_lite = tfma.EvalConfig()
  eval_config_lite.CopyFrom(eval_config)
  # Informs the evaluator that the model is a TFLite model.
  eval_config_lite.model_specs[0].model_type = 'tf_lite'

  # Uses TFMA to compute the evaluation statistics over features of a model.
  evaluator = Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      eval_config=eval_config,
      instance_name='mnist')

  # Uses TFMA to compute the evaluation statistics over features of a TFLite
  # model.
  evaluator_lite = Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer_lite.outputs['model'],
      eval_config=eval_config_lite,
      instance_name='mnist_lite')

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir)),
      instance_name='mnist')

  # Checks whether the TFLite model passed the validation steps and pushes the
  # model to a file destination if check passed.
  pusher_lite = Pusher(
      model=trainer_lite.outputs['model'],
      model_blessing=evaluator_lite.outputs['blessing'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir_lite)),
      instance_name='mnist_lite')

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=[
          example_gen,
          statistics_gen,
          schema_gen,
          example_validator,
          transform,
          trainer,
          trainer_lite,
          evaluator,
          evaluator_lite,
          pusher,
          pusher_lite,
      ],
      enable_cache=True,
      metadata_connection_config=metadata.sqlite_metadata_connection_config(
          metadata_path),
      beam_pipeline_args=beam_pipeline_args)

In [18]:
# To run this pipeline from the python CLI:
#   $python mnist_pipeline_native_keras.py
if __name__ == '__main__':
  absl.logging.set_verbosity(absl.logging.INFO)
  BeamDagRunner().run(
      _create_pipeline(
          pipeline_name=_pipeline_name,
          pipeline_root=_pipeline_root,
          data_root=_data_root,
          module_file=_module_file,
          module_file_lite=_module_file_lite,
          serving_model_dir=_serving_model_dir,
          serving_model_dir_lite=_serving_model_dir_lite,
          metadata_path=_metadata_path,
          beam_pipeline_args=_beam_pipeline_args))

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.


Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.


INFO:absl:Component ImportExampleGen depends on [].
INFO:absl:Component ImportExampleGen is scheduled.
INFO:absl:Component StatisticsGen depends on ['Run[ImportExampleGen]'].
INFO:absl:Component StatisticsGen is scheduled.
INFO:absl:Component SchemaGen depends on ['Run[StatisticsGen]'].
INFO:absl:Component SchemaGen is scheduled.
INFO:absl:Component ExampleValidator depends on ['Run[StatisticsGen]', 'Run[SchemaGen]'].
INFO:absl:Component ExampleValidator is scheduled.
INFO:absl:Component Transform depends on ['Run[ImportExampleGen]', 'Run[SchemaGen]'].
INFO:absl:Component Transform is scheduled.
INFO:absl:Component Trainer.mnist depends on ['Run[Transform]', 'Run[SchemaGen]'].
INFO:absl:Component Trainer.mnist is scheduled.
INFO:absl:Component Trainer.mnist_lite depends on ['Run[Transform]', 'Run[SchemaGen]'].
INFO:absl:Component Trainer.mnist_lite is scheduled.
INFO:absl:Component Evaluator.mnist depends on ['Run[Trainer.mnist]', 'Run[ImportExampleGen]'].
INFO:absl:Component Evaluator

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /content/drive/MyDrive/Fall_2020/Advanced Deep Learning- CMPE 297- 49/Assignments/Assignment 6_TFX interactivecontext based colab/tfx/pipelines/mnist_native_keras/SchemaGen/schema/10/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component SchemaGen is finished.
INFO:absl:Component Transform is running.
INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Attempting to infer TFX Python dependency for beam
INFO:absl:Copying all content from install dir /usr/local/lib/python3.6/dist-packages/tfx to temp dir /tmp/tmp4nugb1bs/build/tfx
INFO:absl:Generating a temp setup file at /tmp/tmp4nugb1bs/build/tfx/setup.py
INFO:absl:Creating temporary sdist package, l

...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Fall_2020/Advanced Deep Learning- CMPE 297- 49/Assignments/Assignment 6_TFX interactivecontext based colab/tfx/pipelines/mnist_native_keras/Trainer.mnist_lite/model/15/serving_model_dir/temp/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Fall_2020/Advanced Deep Learning- CMPE 297- 49/Assignments/Assignment 6_TFX interactivecontext based colab/tfx/pipelines/mnist_native_keras/Trainer.mnist_lite/model/15/serving_model_dir/temp/assets


INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False


INFO:absl:Training complete. Model written to /content/drive/MyDrive/Fall_2020/Advanced Deep Learning- CMPE 297- 49/Assignments/Assignment 6_TFX interactivecontext based colab/tfx/pipelines/mnist_native_keras/Trainer.mnist_lite/model/15/serving_model_dir. ModelRun written to /content/drive/MyDrive/Fall_2020/Advanced Deep Learning- CMPE 297- 49/Assignments/Assignment 6_TFX interactivecontext based colab/tfx/pipelines/mnist_native_keras/Trainer.mnist_lite/model_run/15
INFO:absl:Running publisher for Trainer.mnist_lite
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Trainer.mnist_lite is finished.
INFO:absl:Component Evaluator.mnist_lite is running.
INFO:absl:Running driver for Evaluator.mnist_lite
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator.mnist_lite
INFO:absl:Attempting to infer TFX Python dependency for beam
INFO:absl:Copying all content from install dir /usr/local/lib/python3.6/dist-packages/tfx to temp di